In [3]:
ord("兰")

20848

In [4]:
[ord(x) for x in "复旦大学"]

[22797, 26086, 22823, 23398]

In [6]:
list("复旦大学".encode("utf32"))

[255, 254, 0, 0, 13, 89, 0, 0, 230, 101, 0, 0, 39, 89, 0, 0, 102, 91, 0, 0]

In [9]:
tokens = list("复旦大学是中国上海的一所顶尖综合性大学，成立于1905年。它以其在学术研究、教学质量和社会影响力方面的卓越表现而著称。复旦大学拥有众多学科优势，尤其在文学、法学、商学、医学、计算机科学等领域享有盛誉。它也是中国“211工程”和“985工程”重点支持的大学之一。    你对复旦大学有兴趣吗？是考虑申请还是想了解某些具体方面的信息？".encode("utf8"))

# byte pair
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

stats = get_stats(tokens)
print(sorted(((v, k) for k, v in stats.items()), reverse=True))

[(13, (229, 173)), (13, (173, 166)), (9, (229, 164)), (9, (227, 128)), (6, (167, 229)), (5, (230, 156)), (5, (228, 184)), (5, (164, 167)), (5, (128, 129)), (4, (239, 188)), (4, (231, 154)), (4, (230, 152)), (4, (229, 133)), (4, (228, 186)), (4, (226, 128)), (4, (166, 230)), (4, (166, 227)), (4, (154, 132)), (4, (152, 175)), (4, (128, 130)), (3, (231, 167)), (3, (230, 151)), (3, (230, 150)), (3, (228, 188)), (3, (182, 229)), (3, (166, 229)), (3, (164, 141)), (3, (156, 137)), (3, (151, 166)), (3, (141, 230)), (3, (130, 229)), (3, (32, 32)), (2, (233, 157)), (2, (233, 135)), (2, (232, 182)), (2, (232, 128)), (2, (231, 168)), (2, (229, 183)), (2, (229, 176)), (2, (229, 174)), (2, (229, 156)), (2, (229, 155)), (2, (229, 146)), (2, (229, 144)), (2, (229, 138)), (2, (228, 189)), (2, (228, 185)), (2, (188, 159)), (2, (188, 140)), (2, (185, 233)), (2, (184, 173)), (2, (184, 128)), (2, (183, 165)), (2, (175, 230)), (2, (175, 228)), (2, (174, 131)), (2, (173, 229)), (2, (168, 139)), (2, (167, 145

In [10]:
chr(229), chr(173)

('å', '\xad')

In [11]:
top_pair = max(stats, key=stats.get)
top_pair

(229, 173)

In [12]:
def merge(ids, pair, idx):
    newids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] ==pair[0] and ids[i+1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1

    return newids

print(merge([5, 6, 6, 6, 7, 3, 4, 6, 7], (6, 7), 99))

[5, 6, 6, 99, 3, 4, 99]


In [14]:
print(len(tokens))
tokens2 = merge(tokens, top_pair, 256)
print(len(tokens2))

467
454


In [15]:
text = open("tests/taylorswift.txt", "r", encoding="utf-8").read()
tokens = text.encode("utf-8")
tokens = list(map(int, tokens))

In [17]:

def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts


def merge(ids, pair, idx):
    newids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] ==pair[0] and ids[i+1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1

    return newids

vocab_size = 276
num_merges = vocab_size - 256
ids = list(tokens)

merges = {}

for i in range(num_merges):
    stats = get_stats(ids)
    pair = max(stats, key=stats.get)
    idx = 256 + i
    print(f"merging {pair} into a new token {idx}")
    ids = merge(ids, pair, idx)
    merges[pair] = idx


merging (101, 32) into a new token 256
merging (44, 32) into a new token 257
merging (100, 32) into a new token 258
merging (46, 32) into a new token 259
merging (114, 32) into a new token 260
merging (50, 48) into a new token 261
merging (115, 32) into a new token 262
merging (105, 110) into a new token 263
merging (111, 110) into a new token 264
merging (114, 105) into a new token 265
merging (116, 32) into a new token 266
merging (116, 104) into a new token 267
merging (101, 258) into a new token 268
merging (257, 261) into a new token 269
merging (97, 110) into a new token 270
merging (97, 114) into a new token 271
merging (101, 260) into a new token 272
merging (121, 32) into a new token 273
merging (97, 108) into a new token 274
merging (267, 256) into a new token 275


In [18]:
print("tokens length", len(tokens))
print("ids length", len(ids))

tokens length 185768
ids length 147440


In [20]:
# decoding
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

def decode(ids):
    tokens = b"".join(vocab[idx] for idx in ids)
    text = tokens.decode("utf-8", errors='replace')
    return text

print(decode([128]))

�


In [23]:
def encode(text):
    tokens = list(text.encode("utf-8"))
    while len(tokens) >= 2:
        stats = get_stats((tokens))
        pair = min(stats, key=lambda p : merges.get(p, float("inf")))
        if pair not in merges:
            break
        idx = merges[pair]
        tokens = merge(tokens, pair, idx)
    return tokens


print(encode("hello world!"))

[104, 101, 108, 108, 111, 32, 119, 111, 114, 108, 100, 33]


In [25]:
print(decode(encode("helloworldfswaf")))

helloworldfswaf


In [27]:
text2 = decode(encode(text))
print(text == text2)

True


In [28]:
import tiktoken

enc = tiktoken.get_encoding("gpt2")
print(enc.encode("   hello"))

enc = tiktoken.get_encoding("cl100k_base")
print(enc.encode("   hello"))

[220, 220, 23748]
[256, 24748]


In [ ]:
import sentencepiece as spm